#Conteúdo do app.py (Streamlit)

In [29]:
%%writefile app.py
import os
import json
from collections import Counter

import pandas as pd
import streamlit as st

# ==========================
# 1. Caminhos principais
# ==========================
BASE_DIR = "/content/drive/MyDrive/Big Data/Trab Final BigData"
DATA_DIR = f"{BASE_DIR}/dados_brutos"
PROCESSED_DIR = f"{BASE_DIR}/dados_processados"
MODEL_DIR = f"{BASE_DIR}/modelos"
METRICS_DIR = f"{BASE_DIR}/metricas"

# Arquivos processados
MOVIES_PANDAS_PATH = os.path.join(PROCESSED_DIR, "movies_parquet_pandas.parquet")
RATINGS_PANDAS_PATH = os.path.join(PROCESSED_DIR, "ratings_parquet_pandas.parquet")  # opcional no app
TOP_MOVIES_PATH = os.path.join(PROCESSED_DIR, "top_movies.parquet")
USER_RECS_PARQUET = os.path.join(PROCESSED_DIR, "user_recs_600.parquet")
USER_RECS_PARQUET_FLAT = os.path.join(PROCESSED_DIR, "user_recs_600_flat.parquet")


# Métricas e imagens
ALS_METRICS_PATH = os.path.join(METRICS_DIR, "als_metrics.json")
NULL_COUNTS_PATH = os.path.join(METRICS_DIR, "null_counts.csv")
FILMES_POR_GENERO_IMG = os.path.join(METRICS_DIR, "filmes_por_genero.png")
NOTA_MEDIA_POR_GENERO_IMG = os.path.join(METRICS_DIR, "nota_media_por_genero.png")
DISTRIBUICAO_NOTAS_IMG = os.path.join(METRICS_DIR, "distribuicao_notas.png")

st.set_page_config(
    page_title="Sistema de Recomendação - MovieLens",
    layout="wide"
)

# ==========================
# 2. Funções de carregamento
# ==========================

@st.cache_data
def load_movies() -> pd.DataFrame:
    """Carrega apenas a tabela de filmes (versão pandas)."""
    if os.path.exists(MOVIES_PANDAS_PATH):
        return pd.read_parquet(MOVIES_PANDAS_PATH)
    # fallback: tentar parquet "normal" ou CSV
    movies_parquet = os.path.join(PROCESSED_DIR, "movies_parquet")
    movies_csv = os.path.join(BASE_DIR, "ml-latest", "movies.csv")

    if os.path.exists(movies_parquet):
        return pd.read_parquet(movies_parquet)
    elif os.path.exists(movies_csv):
        return pd.read_csv(movies_csv)
    else:
        return pd.DataFrame()


@st.cache_data
def load_top_movies() -> pd.DataFrame | None:
    """Carrega top_movies.parquet (já agregado por filme)."""
    if os.path.exists(TOP_MOVIES_PATH):
        return pd.read_parquet(TOP_MOVIES_PATH)
    return None


@st.cache_data
def load_als_metrics() -> dict | None:
    """Carrega métricas do modelo ALS (RMSE, MAPE) do JSON."""
    if not os.path.exists(ALS_METRICS_PATH):
        return None
    try:
        with open(ALS_METRICS_PATH, "r") as f:
            return json.load(f)
    except Exception:
        return None


@st.cache_data
def load_null_counts() -> pd.DataFrame | None:
    """Carrega tabela de nulos calculada no back."""
    if not os.path.exists(NULL_COUNTS_PATH):
        return None
    try:
        return pd.read_csv(NULL_COUNTS_PATH)
    except Exception:
        return None


@st.cache_data
def load_dataset_summary() -> dict | None:
    """
    Usamos top_movies.parquet para ter:
    - total de ratings
    - nota média global (ponderada por num_ratings)
    - total de filmes (usando movies_parquet_pandas, se existir)
    """
    movies_df = load_movies()
    top_movies_df = load_top_movies()

    if top_movies_df is None or top_movies_df.empty:
        return None

    # Garante nomes coerentes
    cols = list(top_movies_df.columns)
    # tenta achar num_ratings e avg_rating
    if "num_ratings" not in cols or "avg_rating" not in cols:
        # tentar detectar pelos nomes
        # (se veio do spark, deve estar certo, então aqui é só safety)
        return None

    total_ratings = int(top_movies_df["num_ratings"].sum())
    if total_ratings == 0:
        return None

    # média global ponderada
    num = (top_movies_df["avg_rating"] * top_movies_df["num_ratings"]).sum()
    avg_global = float(num / total_ratings)

    if not movies_df.empty and "movieId" in movies_df.columns:
        total_movies = int(movies_df["movieId"].nunique())
    elif "movieId" in top_movies_df.columns:
        total_movies = int(top_movies_df["movieId"].nunique())
    else:
        total_movies = None

    return {
        "total_ratings": total_ratings,
        "total_movies": total_movies,
        "avg_rating_global": avg_global,
        "rating_min": 0.5,
        "rating_max": 5.0,
    }


@st.cache_data
def load_top_tables() -> dict:
    """
    Carrega top_movies.parquet e deriva:
      - top_movies_full
      - top10_most_watched
      - top10_best_rated
    Tudo em pandas.
    """
    top_movies_df = load_top_movies()
    movies_df = load_movies()

    tables: dict[str, pd.DataFrame] = {}

    if top_movies_df is None or top_movies_df.empty:
        return tables

    df = top_movies_df.copy()

    # Se por acaso não tiver título/gênero, faz merge com movies_df
    if "title" not in df.columns or "genres" not in df.columns:
        if not movies_df.empty and "movieId" in movies_df.columns:
            df = df.merge(
                movies_df[["movieId", "title", "genres"]],
                on="movieId",
                how="left"
            )

    # Tipagens
    if "num_ratings" in df.columns:
        df["num_ratings"] = df["num_ratings"].astype("int64", errors="ignore")
    if "avg_rating" in df.columns:
        df["avg_rating"] = df["avg_rating"].astype("float64", errors="ignore")

    tables["top_movies_full"] = df

    # Top 10 mais assistidos
    if "num_ratings" in df.columns and "avg_rating" in df.columns:
        df_pop = df.sort_values(
            by=["num_ratings", "avg_rating"],
            ascending=[False, False]
        ).head(10)
        tables["top10_most_watched"] = df_pop

        # Top 10 melhor nota (com mínimo de avaliações)
        min_ratings = 50
        df_best = df[df["num_ratings"] >= min_ratings].sort_values(
            by=["avg_rating", "num_ratings"],
            ascending=[False, False]
        ).head(10)
        tables["top10_best_rated"] = df_best

    return tables

@st.cache_data
def load_precomputed_recommendations() -> pd.DataFrame | None:
    """
    Carrega user_recs_600_flat.parquet (gerado no back com Spark),
    já no formato (userId, movieId, predicted_rating), e faz merge
    com a tabela de filmes em pandas.
    """
    movies_df = load_movies()

    if not os.path.exists(USER_RECS_PARQUET_FLAT):
        st.warning(
            "Arquivo `user_recs_600_flat.parquet` não encontrado em `dados_processados/`.\n"
            "Rode o notebook de back-end para gerar as recomendações flatten."
        )
        return None

    try:
        recs_flat = pd.read_parquet(USER_RECS_PARQUET_FLAT, engine="pyarrow")
    except Exception as e:
        st.error(f"Erro ao ler `user_recs_600_flat.parquet`: {e}")
        return None

    if recs_flat.empty:
        st.warning("`user_recs_600_flat.parquet` foi carregado, mas está vazio.")
        return None

    # Garante nomes padrão
    df = recs_flat.copy()
    if "rating" in df.columns and "predicted_rating" not in df.columns:
        df = df.rename(columns={"rating": "predicted_rating"})

    # Tipos
    df["userId"] = df["userId"].astype("int64")
    df["movieId"] = df["movieId"].astype("int64")
    df["predicted_rating"] = df["predicted_rating"].astype("float64")

    # Merge com filmes
    if not movies_df.empty and "movieId" in movies_df.columns:
        df = df.merge(
            movies_df[["movieId", "title", "genres"]],
            on="movieId",
            how="left"
        )

    # Só pra debug: quantos users e linhas
    st.write(f"Recomendações flatten carregadas: {df.shape[0]} linhas, {df['userId'].nunique()} usuários.")
    return df

# ==========================
# 3. Lógica de recomendação
# ==========================

def get_user_recommendations_pandas(
    user_id: int,
    recs_flat_df: pd.DataFrame,
    num_recs: int = 10
) -> tuple[pd.DataFrame | None, dict | None]:

    if recs_flat_df is None or recs_flat_df.empty:
        return None, None

    user_id = int(user_id)
    recs_user = recs_flat_df[recs_flat_df["userId"] == user_id].copy()

    if recs_user.empty:
        return None, None

    # Garante unicidade por filme (só por segurança)
    recs_user = recs_user.drop_duplicates(subset=["movieId"])

    recs_user = recs_user.sort_values(
        by="predicted_rating",
        ascending=False
    ).head(num_recs)


    # Descobrir gênero dominante nas recomendações (só olhando os filmes recomendados)
    if "genres" in recs_user.columns:
        genres_series = recs_user["genres"].dropna().astype(str).str.split("|")
        genres_flat = [
            g.strip()
            for sub in genres_series
            for g in (sub if isinstance(sub, list) else [])
            if g.strip() and g.strip() != "(no genres listed)"
        ]
        if genres_flat:
            counts = Counter(genres_flat)
            fav_genre, fav_count = counts.most_common(1)[0]
            total = sum(counts.values())
            genre_info = {
                "favorite_genre": fav_genre,
                "favorite_genre_percent": fav_count / total * 100.0,
            }
        else:
            genre_info = None
    else:
        genre_info = None

    return recs_user, genre_info


# ==========================
# 4. Páginas do Streamlit
# ==========================

def page_recommendations():
    st.title("🎬 Sistema de Recomendação (Pré-cálculo ALS)")

    recs_flat = load_precomputed_recommendations()
    top_tables = load_top_tables()

    if recs_flat is None or recs_flat.empty:
        st.warning(
            "Nenhuma recomendação pré-calculada encontrada em "
            "`dados_processados/user_recs_600.parquet`.\n\n"
            "Rode o notebook de **back-end** para gerar as previsões "
            "para ~500/600 usuários."
        )
        return

    # Usuários disponíveis (somente os pré-calculados)
    user_ids = sorted(recs_flat["userId"].unique())
    st.markdown("#### Selecione um usuário (entre os pré-calculados)")
    user_id = st.selectbox("userId", user_ids, index=0)

    num_recs = st.slider(
        "Quantidade de recomendações",
        min_value=5,
        max_value=50,
        step=5,
        value=10
    )

    if st.button("Gerar recomendações"):
        recs_df, genre_info = get_user_recommendations_pandas(
            user_id=user_id,
            recs_flat_df=recs_flat,
            num_recs=num_recs
        )

        if recs_df is None:
            st.warning(
                f"Usuário {user_id} não está na lista de usuários "
                "com recomendações pré-calculadas."
            )
        else:
            st.markdown(f"### Recomendações para o usuário `{user_id}`")
            st.dataframe(
                recs_df[["movieId", "title", "genres", "predicted_rating"]],
                use_container_width=True
            )

            if genre_info is not None:
                st.info(
                    f"Gênero dominante nas recomendações: "
                    f"**{genre_info['favorite_genre']}** "
                    f"(~{genre_info['favorite_genre_percent']:.1f}% dos filmes recomendados)."
                )

    st.markdown("---")
    st.markdown("### 🌍 Top 10 filmes mais assistidos (global)")

    if "top10_most_watched" in top_tables:
        st.dataframe(
            top_tables["top10_most_watched"][
                ["movieId", "title", "genres", "num_ratings", "avg_rating"]
            ],
            use_container_width=True
        )
    else:
        st.info(
            "Tabela de filmes mais assistidos não disponível "
            "(verifique se `top_movies.parquet` existe em dados_processados/)."
        )


def page_dashboard():
    st.title("📊 Dashboard Analítico e Qualidade dos Dados")

    dataset_summary = load_dataset_summary()
    top_tables = load_top_tables()
    als_metrics = load_als_metrics()
    null_counts_df = load_null_counts()

    # ---- Métricas principais do dataset ----
    st.markdown("### Visão geral do dataset")

    if dataset_summary is not None:
        c1, c2, c3, c4 = st.columns(4)
        if dataset_summary.get("total_movies") is not None:
            c1.metric("Total de filmes", dataset_summary["total_movies"])
        else:
            c1.metric("Total de filmes", "N/D")
        c2.metric("Total de avaliações", dataset_summary["total_ratings"])
        c3.metric(
            "Nota média global",
            f"{dataset_summary['avg_rating_global']:.2f}"
        )
        c4.metric(
            "Faixa de notas",
            f"{dataset_summary['rating_min']} – {dataset_summary['rating_max']}"
        )
    else:
        st.info(
            "Não foi possível montar o resumo do dataset. "
            "Verifique se `top_movies.parquet` existe em dados_processados/."
        )

    # ---- Top 10 melhor avaliados ----
    st.markdown("### ⭐ Top 10 filmes com melhor nota (com mínimo de avaliações)")
    if "top10_best_rated" in top_tables:
        st.dataframe(
            top_tables["top10_best_rated"][
                ["movieId", "title", "genres", "num_ratings", "avg_rating"]
            ],
            use_container_width=True
        )
    else:
        st.info("Tabela de filmes melhor avaliados não disponível.")

    st.markdown("---")
    st.markdown("### 🔍 Qualidade dos dados (nulos)")

    if null_counts_df is not None and not null_counts_df.empty:
        st.dataframe(null_counts_df, use_container_width=True)
    else:
        st.info(
            "Arquivo `null_counts.csv` não encontrado em `metricas/` "
            "ou está vazio."
        )

    st.markdown("---")
    st.markdown("### 📐 Métricas do modelo ALS (offline)")

    if als_metrics is not None:
        st.json(als_metrics)
    else:
        st.info(
            "Arquivo `als_metrics.json` não encontrado em `metricas/`. "
            "Rode o notebook de back-end para gerar RMSE e MAPE."
        )

    st.markdown("---")
    st.markdown("### 📈 Visualizações (geradas no back-end)")

    col1, col2 = st.columns(2)
    with col1:
        if os.path.exists(FILMES_POR_GENERO_IMG):
            st.image(
                FILMES_POR_GENERO_IMG,
                caption="Filmes por Gênero",
                use_container_width=True
            )
        else:
            st.info("Imagem `filmes_por_genero.png` não encontrada em `metricas/`.")

        if os.path.exists(NOTA_MEDIA_POR_GENERO_IMG):
            st.image(
                NOTA_MEDIA_POR_GENERO_IMG,
                caption="Nota Média por Gênero",
                use_container_width=True
            )
        else:
            st.info("Imagem `nota_media_por_genero.png` não encontrada em `metricas/`.")

    with col2:
        if os.path.exists(DISTRIBUICAO_NOTAS_IMG):
            st.image(
                DISTRIBUICAO_NOTAS_IMG,
                caption="Distribuição das Notas",
                use_container_width=True
            )
        else:
            st.info(
                "Imagem `distribuicao_notas.png` não encontrada em `metricas/`."
            )


# ==========================
# 5. Main
# ==========================

def main():
    st.sidebar.title("Navegação")
    page = st.sidebar.radio(
        "Escolha a página:",
        ("Recomendações", "Dashboard Analítico")
    )

    if page == "Recomendações":
        page_recommendations()
    else:
        page_dashboard()


if __name__ == "__main__":
    main()


Overwriting app.py


#3) Ngrok + execução do Streamlit

In [30]:
# ============================================================
# SEÇÃO 3 – Expor o Streamlit via Ngrok (em Colab)
# ============================================================
import os
from pyngrok import ngrok

# IMPORTANTE: substitua pela sua chave
NGROK_AUTH_TOKEN = "33t9PRrLLOGGkZ5AStHNhaOq0zV_4v8RRxZArbssi3HCYZZMi"
os.environ["NGROK_AUTHTOKEN"] = NGROK_AUTH_TOKEN

# Fecha túneis anteriores
!pkill -f ngrok 2>/dev/null || echo "Nenhum ngrok em execução."
!pkill -f streamlit || echo "Nenhum processo Streamlit em execução."

public_url = ngrok.connect(8501).public_url
print(f"🚀 URL pública do app: {public_url}")

# Executa o app Streamlit
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > streamlit.log 2>&1 &
# !streamlit run app.py --server.port 8501 --server.address 0.0.0.0 >/dev/null 2>&1 &

^C
^C
🚀 URL pública do app: https://unpathetic-doubtlessly-alvina.ngrok-free.dev
